# Посещение врачей

In [1]:
import pandas as pd
import numpy as np

# игнорирование предупреждения о формате при импорте из excel
import warnings

warnings.simplefilter("ignore")

%load_ext lab_black

ISO-8859-1

# Если нестандартные кодировки:

In [313]:
# загрузка файла для ЕАО !!!
df_data = pd.read_csv(
    "/home/sk27/work/med/data/source.csv",
    sep=",",
    encoding="utf-16",
    low_memory=False,
)

year,month,region,clinic,oid,depart,branch,doctor_specialty,doctor_profile,terms,doctor_id,diagnosis,purpose,service_id,usl_service_id,usl_doctor_specialty,usl_clinic,usl_depart,sl_id,code_usl,kol_usl,yet_sum

## Заголовок добавить вручную
### Переходить сразу к расшифровке кодов

In [314]:
df_data

,year,month,region,oktmo,clinic,oid,depart,branch,doctor_specialty,doctor_profile,...,purpose,service_id,usl_service_id,usl_doctor_specialty,usl_clinic,usl_depart,sl_id,code_usl,kol_usl,yet_sum
0,2023,1,99,99000000000,790001,1.2.643.5.1.13.13.12.2.79.8850,79000100,NaN,24,28,...,1.0,B01.014.001,137-097-107 64,24,790001,79000100,1,1,1,NaN
1,2023,1,99,99000000000,790001,1.2.643.5.1.13.13.12.2.79.8850,79000100,NaN,24,28,...,1.3,R03.03.102.001,137-097-107 64,24,790001,79000100,1,1,1,NaN
2,2023,1,99,99000000000,790001,1.2.643.5.1.13.13.12.2.79.8850,79000100,NaN,24,28,...,1.0,B01.014.001,137-097-107 64,24,790001,79000100,10,1,1,NaN
3,2023,1,99,99000000000,790001,1.2.643.5.1.13.13.12.2.79.8850,79000100,NaN,24,28,...,3.0,B01.014.002,137-097-107 64,24,790001,79000100,12,5,1,NaN
4,2023,1,99,99000000000,790001,1.2.643.5.1.13.13.12.2.79.8850,79000100,NaN,24,28,...,3.0,B01.014.002,138-993-968 45,24,790001,79000100,2,15,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25405,2023,2,99,99000000000,790021,1.2.643.5.1.13.13.12.2.79.8838,79002100,NaN,41,60,...,1.0,B01.027.001,17717264189,41,790021,79002100,2,3,1,NaN
25406,2023,2,99,99000000000,790021,1.2.643.5.1.13.13.12.2.79.8838,79002100,NaN,45,162,...,1.0,B01.028.001,13109077827,45,790021,79002100,1,1,1,NaN
25407,2023,2,99,99000000000,790021,1.2.643.5.1.13.13.12.2.79.8838,79002100,NaN,76,97,...,1.0,B01.047.001,6180038532,76,790021,79002100,1,1,1,NaN
25408,2023,2,99,99000000000,790021,1.2.643.5.1.13.13.12.2.79.8838,79002100,NaN,76,97,...,1.0,B01.047.001,6180038532,76,790021,79002100,1,1,1,NaN


In [315]:
# количество строк и столбцов
df_data.shape

(25410, 23)

In [316]:
# повторяющиеся строки
df_data[df_data.duplicated()]

,year,month,region,oktmo,clinic,oid,depart,branch,doctor_specialty,doctor_profile,...,purpose,service_id,usl_service_id,usl_doctor_specialty,usl_clinic,usl_depart,sl_id,code_usl,kol_usl,yet_sum


In [317]:
# количество повторяющихся строк
df_data[df_data.duplicated()].shape

(0, 23)

In [318]:
# замена истинно постых значений словом nan
# это нужно для подсчёта повторяющихся строк
df_data = df_data.fillna("nan")
# после этого переходим к подсчёту повторяющихся строк

# Обычные кодировки
## 1. Подготовка данных

In [269]:
%%time

# вспомогательные функции

def oktmo(o):
    """
    функция дополняет код ОКТМО до 11 символов или до 10 символов без ведущего 0
    """
    if len(str(o)) == 1:
        l = 9
    else:
        l = 11-len(str(o))
    return int(str(o) + "0"*(l))
    
def visit_count(row):
    """
    функция определяет количество посещений согласно условиям
    """
    if row["purpose"] >= 3:
        res = row["code_usl"] * row["kol_usl"]
    else:
        res = row["sl_id"]
    if np.isnan(res):
        return "не определено"
    else:
        return res
    
# добавление в файл данных строки заголовков
with open("/home/sk27/work/med/data/source.csv", "r") as f:
    first_line = next(f).strip()
    if first_line[0:4] != "year":
        with open(
            "/home/sk27/work/med/data/source.csv", "r", encoding="ISO-8859-1"
        ) as original:
            data = original.read()
        with open("/home/sk27/work/med/data/source.csv", "w") as modified:
            modified.write(
                "year;month;region;clinic;oid;depart;branch;doctor_specialty;doctor_profile;terms;doctor_id;diagnosis;purpose;service_id;usl_service_id;usl_doctor_specialty;usl_clinic;usl_depart;sl_id;code_usl;kol_usl;yet_sum\n"
                + data
            )
    else:
        pass

# загрузка файла
df_data = pd.read_csv(
    "/home/sk27/work/med/data/source.csv",
    sep=";",
    encoding="ISO-8859-1",
    low_memory=False,
)


# удаление кавычек
for i, col in enumerate(df_data.columns):
    df_data.iloc[:, i] = df_data.iloc[:, i].astype("str").str.replace('"', "")
# df_data = df_data.apply(pd.to_numeric, errors="ignore")

# замена ВСЕХ запятых точками
for i, col in enumerate(df_data.columns):
    df_data.iloc[:, i] = df_data.iloc[:, i].astype("str").str.replace(',', ".")
df_data = df_data.apply(pd.to_numeric, errors="ignore")

# вставка столбца ОКТМО
df_data.insert(3, "oktmo", df_data["region"].apply(oktmo))

# проверка загруженных данных

# а) проверка количества столбцов - их должно быть именно 23 вместе с добавленным мной столбцом ОКТМО
if df_data.shape[1] > 23:
    print(f"Количество полей в файле (их {df_data.shape[1]}) не соответствует форме отчёта, должно быть 23.")
else:
    pass

# б) проверка столбца terms на "только 3"
try:
    if df_data["terms"].mean() != 3:
        print(
            "Нарушено требование об указании в поле 'Условия оказания медпомощи' только значения '3'."
        )
except:
    print("Не проходит проверка на '3'")
    
# расшифровка кодов

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte

In [319]:
# контроль набора данных
df_data

,year,month,region,oktmo,clinic,oid,depart,branch,doctor_specialty,doctor_profile,...,purpose,service_id,usl_service_id,usl_doctor_specialty,usl_clinic,usl_depart,sl_id,code_usl,kol_usl,yet_sum
0,2023,1,99,99000000000,790001,1.2.643.5.1.13.13.12.2.79.8850,79000100,nan,24,28,...,1.0,B01.014.001,137-097-107 64,24,790001,79000100,1,1,1,nan
1,2023,1,99,99000000000,790001,1.2.643.5.1.13.13.12.2.79.8850,79000100,nan,24,28,...,1.3,R03.03.102.001,137-097-107 64,24,790001,79000100,1,1,1,nan
2,2023,1,99,99000000000,790001,1.2.643.5.1.13.13.12.2.79.8850,79000100,nan,24,28,...,1.0,B01.014.001,137-097-107 64,24,790001,79000100,10,1,1,nan
3,2023,1,99,99000000000,790001,1.2.643.5.1.13.13.12.2.79.8850,79000100,nan,24,28,...,3.0,B01.014.002,137-097-107 64,24,790001,79000100,12,5,1,nan
4,2023,1,99,99000000000,790001,1.2.643.5.1.13.13.12.2.79.8850,79000100,nan,24,28,...,3.0,B01.014.002,138-993-968 45,24,790001,79000100,2,15,1,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25405,2023,2,99,99000000000,790021,1.2.643.5.1.13.13.12.2.79.8838,79002100,nan,41,60,...,1.0,B01.027.001,17717264189,41,790021,79002100,2,3,1,nan
25406,2023,2,99,99000000000,790021,1.2.643.5.1.13.13.12.2.79.8838,79002100,nan,45,162,...,1.0,B01.028.001,13109077827,45,790021,79002100,1,1,1,nan
25407,2023,2,99,99000000000,790021,1.2.643.5.1.13.13.12.2.79.8838,79002100,nan,76,97,...,1.0,B01.047.001,6180038532,76,790021,79002100,1,1,1,nan
25408,2023,2,99,99000000000,790021,1.2.643.5.1.13.13.12.2.79.8838,79002100,nan,76,97,...,1.0,B01.047.001,6180038532,76,790021,79002100,1,1,1,nan


In [320]:
# сохранение данных о количестве срок в исходном наборе
s1 = df_data.shape[0]
# сохранение данных о количестве уникальных строк
s2 = df_data.drop_duplicates().shape[0]

## 2. Сколько раз каждая строка повторяется в наборе
#### Добавляем атрибут size, значение которого равно количеству повторов строки в наборе данных

In [321]:
# подчёт, сколько раз строка повторяется в наборе
# при этом набор данных "свёртывается", каждая срока повторяется только один раз, но в столбце size отображается количество повторов
df_data_size = df_data.groupby(df_data.columns.tolist(), as_index=False).size()
print(
    f"Количество уникальных строк в исходном наборе данных: {s2}, количество строк после 'свёртывания' набора: {df_data_size.shape[0]}"
)

Количество уникальных строк в исходном наборе данных: 25410, количество строк после 'свёртывания' набора: 25410


In [322]:
# "развёртывание" набора: повторение каждой строки столько раз, сколько указано в столбце size
df_data = df_data_size.loc[df_data_size.index.repeat(df_data_size["size"])]
df_data.reset_index(drop=True, inplace=True)

In [323]:
# контроль набора данных
df_data

,year,month,region,oktmo,clinic,oid,depart,branch,doctor_specialty,doctor_profile,...,service_id,usl_service_id,usl_doctor_specialty,usl_clinic,usl_depart,sl_id,code_usl,kol_usl,yet_sum,size
0,2023,1,99,99000000000,790001,1.2.643.5.1.13.13.12.2.79.8850,79000100,nan,24,28,...,B01.014.001,137-097-107 64,24,790001,79000100,1,1,1,nan,1
1,2023,1,99,99000000000,790001,1.2.643.5.1.13.13.12.2.79.8850,79000100,nan,24,28,...,B01.014.001,137-097-107 64,24,790001,79000100,1,3,1,nan,1
2,2023,1,99,99000000000,790001,1.2.643.5.1.13.13.12.2.79.8850,79000100,nan,24,28,...,B01.014.002,137-097-107 64,24,790001,79000100,1,3,1,nan,1
3,2023,1,99,99000000000,790001,1.2.643.5.1.13.13.12.2.79.8850,79000100,nan,24,28,...,B01.014.001,137-097-107 64,24,790001,79000100,1,1,1,nan,1
4,2023,1,99,99000000000,790001,1.2.643.5.1.13.13.12.2.79.8850,79000100,nan,24,28,...,R03.03.102.001,137-097-107 64,24,790001,79000100,1,1,1,nan,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25405,2023,2,99,99000000000,790021,1.2.643.5.1.13.13.12.2.79.8838,79002100,nan,206,42,...,26.73,7044889275,206,790021,79002100,20,1,1,nan,1
25406,2023,2,99,99000000000,790021,1.2.643.5.1.13.13.12.2.79.8838,79002100,nan,206,42,...,26.73,7044889275,206,790021,79002100,4,1,1,nan,1
25407,2023,2,99,99000000000,790021,1.2.643.5.1.13.13.12.2.79.8838,79002100,nan,206,42,...,26.73,7044889275,206,790021,79002100,117,1,1,nan,1
25408,2023,2,99,99000000000,790021,1.2.643.5.1.13.13.12.2.79.8838,79002100,nan,206,42,...,26.73,7044889275,206,790021,79002100,1,1,1,nan,1


In [324]:
if df_data.shape[0] == s1:
    print("Число строк не изменилось")
else:
    print("Внимание! Изменилось число строк!")

Число строк не изменилось


In [325]:
df_data.shape[0]

25410

## 3. Расшифровка кодов

In [48]:
%%time

# загрузка справочников
f_032 = pd.read_excel("/home/sk27/work/med/data/F032.xlsx")
v_021 = pd.read_excel("/home/sk27/work/med/data/V021.xlsx")
v_002 = pd.read_excel("/home/sk27/work/med/data/V002.xlsx")
v_006 = pd.read_excel("/home/sk27/work/med/data/V006.xlsx")
v_025 = pd.read_excel("/home/sk27/work/med/data/V025.xlsx")
oid_data = pd.read_excel("/home/sk27/work/med/data/oid_data.xlsx")

# актуализация справочников
f_032 = f_032[["OKTMO_P", "SUBJ"]].drop_duplicates().reset_index(drop=True)
v_021 = v_021[v_021["DATEEND"].isna()]
v_002 = v_002[v_002["DATEEND"].isna()]
v_006 = v_006[v_006["DATEEND"].isna()]
v_025 = v_025[v_025["DATEEND"].isna()]

CPU times: user 47.7 s, sys: 171 ms, total: 47.9 s
Wall time: 48 s


In [326]:
# расшифорвка названия субьекта Росссийской Федерации
df_data = df_data.merge(f_032, left_on="oktmo", right_on="OKTMO_P", how="left")

In [327]:
df_data = df_data[
    [
        "year",
        "month",
        "region",
        "oktmo",
        "SUBJ",
        "clinic",
        "oid",
        "depart",
        "branch",
        "doctor_specialty",
        "doctor_profile",
        "terms",
        "doctor_id",
        "diagnosis",
        "purpose",
        "service_id",
        "usl_service_id",
        "usl_doctor_specialty",
        "usl_clinic",
        "usl_depart",
        "sl_id",
        "code_usl",
        "kol_usl",
        "yet_sum",
        "size",
    ]
].reset_index(drop=True)
if df_data.shape[0] == s1:
    print("Число строк не изменилось")
else:
    print("Внимание! Изменилось число строк!")

Число строк не изменилось


In [328]:
# расшифровка наименования (сокращённого) медучреждения
df_data = df_data.merge(oid_data, left_on="oid", right_on="oid", how="left")

In [329]:
df_data = df_data[
    [
        "year",
        "month",
        "region",
        "oktmo",
        "SUBJ",
        "clinic",
        "oid",
        "nameShort",
        "depart",
        "branch",
        "doctor_specialty",
        "doctor_profile",
        "terms",
        "doctor_id",
        "diagnosis",
        "purpose",
        "service_id",
        "usl_service_id",
        "usl_doctor_specialty",
        "usl_clinic",
        "usl_depart",
        "sl_id",
        "code_usl",
        "kol_usl",
        "yet_sum",
        "size",
    ]
].reset_index(drop=True)
if df_data.shape[0] == s1:
    print("Число строк не изменилось")
else:
    print("Внимание! Изменилось число строк!")

Число строк не изменилось


In [330]:
# расшифровка специальности врача
df_data = df_data.merge(
    v_021, left_on="doctor_specialty", right_on="IDSPEC", how="left"
)

In [331]:
df_data = df_data[
    [
        "year",
        "month",
        "region",
        "oktmo",
        "SUBJ",
        "clinic",
        "oid",
        "nameShort",
        "depart",
        "branch",
        "doctor_specialty",
        "POSTNAME",
        "doctor_profile",
        "terms",
        "doctor_id",
        "diagnosis",
        "purpose",
        "service_id",
        "usl_service_id",
        "usl_doctor_specialty",
        "usl_clinic",
        "usl_depart",
        "sl_id",
        "code_usl",
        "kol_usl",
        "yet_sum",
        "size",
    ]
].reset_index(drop=True)
if df_data.shape[0] == s1:
    print("Число строк не изменилось")
else:
    print("Внимание! Изменилось число строк!")

Число строк не изменилось


In [332]:
# расшифровка профиля врача
df_data = df_data.merge(v_002, left_on="doctor_profile", right_on="IDPR", how="left")

In [333]:
df_data = df_data[
    [
        "year",
        "month",
        "region",
        "oktmo",
        "SUBJ",
        "clinic",
        "oid",
        "nameShort",
        "depart",
        "branch",
        "doctor_specialty",
        "POSTNAME",
        "doctor_profile",
        "PRNAME",
        "terms",
        "doctor_id",
        "diagnosis",
        "purpose",
        "service_id",
        "usl_service_id",
        "usl_doctor_specialty",
        "usl_clinic",
        "usl_depart",
        "sl_id",
        "code_usl",
        "kol_usl",
        "yet_sum",
        "size",
    ]
].reset_index(drop=True)
if df_data.shape[0] == s1:
    print("Число строк не изменилось")
else:
    print("Внимание! Изменилось число строк!")

Число строк не изменилось


In [334]:
# расшифровка специальности врача, оказавшего услугу
df_data = df_data.merge(
    v_021, left_on="doctor_specialty", right_on="IDSPEC", how="left"
)

In [335]:
df_data = df_data[
    [
        "year",
        "month",
        "region",
        "oktmo",
        "SUBJ",
        "clinic",
        "oid",
        "nameShort",
        "depart",
        "branch",
        "doctor_specialty",
        "POSTNAME_x",
        "doctor_profile",
        "PRNAME",
        "terms",
        "doctor_id",
        "diagnosis",
        "purpose",
        "service_id",
        "usl_service_id",
        "usl_doctor_specialty",
        "POSTNAME_y",
        "usl_clinic",
        "usl_depart",
        "sl_id",
        "code_usl",
        "kol_usl",
        "yet_sum",
        "size",
    ]
].reset_index(drop=True)
if df_data.shape[0] == s1:
    print("Число строк не изменилось")
else:
    print("Внимание! Изменилось число строк!")

Число строк не изменилось


In [336]:
# расшифровка условий оказания помощи
df_data = df_data.merge(v_006, left_on="terms", right_on="IDUMP", how="left")

In [337]:
df_data = df_data[
    [
        "year",
        "month",
        "region",
        "oktmo",
        "SUBJ",
        "clinic",
        "oid",
        "nameShort",
        "depart",
        "branch",
        "doctor_specialty",
        "POSTNAME_x",
        "doctor_profile",
        "PRNAME",
        "terms",
        "UMPNAME",
        "doctor_id",
        "diagnosis",
        "purpose",
        "service_id",
        "usl_service_id",
        "usl_doctor_specialty",
        "POSTNAME_y",
        "usl_clinic",
        "usl_depart",
        "sl_id",
        "code_usl",
        "kol_usl",
        "yet_sum",
        "size",
    ]
].reset_index(drop=True)
if df_data.shape[0] == s1:
    print("Число строк не изменилось")
else:
    print("Внимание! Изменилось число строк!")

Число строк не изменилось


In [338]:
# расшифровка цели посещения
df_data = df_data.merge(v_025, left_on="purpose", right_on="IDPC", how="left")

In [339]:
df_data = df_data[
    [
        "year",
        "month",
        "region",
        "oktmo",
        "SUBJ",
        "clinic",
        "oid",
        "nameShort",
        "depart",
        "branch",
        "doctor_specialty",
        "POSTNAME_x",
        "doctor_profile",
        "PRNAME",
        "terms",
        "UMPNAME",
        "doctor_id",
        "diagnosis",
        "purpose",
        "N_PC",
        "service_id",
        "usl_service_id",
        "usl_doctor_specialty",
        "POSTNAME_y",
        "usl_clinic",
        "usl_depart",
        "sl_id",
        "code_usl",
        "kol_usl",
        "yet_sum",
        "size",
    ]
].reset_index(drop=True)
if df_data.shape[0] == s1:
    print("Число строк не изменилось")
else:
    print("Внимание! Изменилось число строк!")

Число строк не изменилось


In [340]:
df_data

,year,month,region,oktmo,SUBJ,clinic,oid,nameShort,depart,branch,...,usl_service_id,usl_doctor_specialty,POSTNAME_y,usl_clinic,usl_depart,sl_id,code_usl,kol_usl,yet_sum,size
0,2023,1,99,99000000000,Еврейская АО,790001,1.2.643.5.1.13.13.12.2.79.8850,ОГБУЗ Областная больница,79000100,nan,...,137-097-107 64,24,врач-инфекционист,790001,79000100,1,1,1,nan,1
1,2023,1,99,99000000000,Еврейская АО,790001,1.2.643.5.1.13.13.12.2.79.8850,ОГБУЗ Областная больница,79000100,nan,...,137-097-107 64,24,врач-инфекционист,790001,79000100,1,3,1,nan,1
2,2023,1,99,99000000000,Еврейская АО,790001,1.2.643.5.1.13.13.12.2.79.8850,ОГБУЗ Областная больница,79000100,nan,...,137-097-107 64,24,врач-инфекционист,790001,79000100,1,3,1,nan,1
3,2023,1,99,99000000000,Еврейская АО,790001,1.2.643.5.1.13.13.12.2.79.8850,ОГБУЗ Областная больница,79000100,nan,...,137-097-107 64,24,врач-инфекционист,790001,79000100,1,1,1,nan,1
4,2023,1,99,99000000000,Еврейская АО,790001,1.2.643.5.1.13.13.12.2.79.8850,ОГБУЗ Областная больница,79000100,nan,...,137-097-107 64,24,врач-инфекционист,790001,79000100,1,1,1,nan,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25405,2023,2,99,99000000000,Еврейская АО,790021,1.2.643.5.1.13.13.12.2.79.8838,"ОГБУЗ ""Онкологический диспансер""",79002100,nan,...,7044889275,206,фельдшер,790021,79002100,20,1,1,nan,1
25406,2023,2,99,99000000000,Еврейская АО,790021,1.2.643.5.1.13.13.12.2.79.8838,"ОГБУЗ ""Онкологический диспансер""",79002100,nan,...,7044889275,206,фельдшер,790021,79002100,4,1,1,nan,1
25407,2023,2,99,99000000000,Еврейская АО,790021,1.2.643.5.1.13.13.12.2.79.8838,"ОГБУЗ ""Онкологический диспансер""",79002100,nan,...,7044889275,206,фельдшер,790021,79002100,117,1,1,nan,1
25408,2023,2,99,99000000000,Еврейская АО,790021,1.2.643.5.1.13.13.12.2.79.8838,"ОГБУЗ ""Онкологический диспансер""",79002100,nan,...,7044889275,206,фельдшер,790021,79002100,1,1,1,nan,1


## 3. Расчёт количества посещений

In [341]:
%%time
# определение количества посещений
df_data["visit_count"] = df_data.apply(visit_count, axis=1)

CPU times: user 431 ms, sys: 11.9 ms, total: 443 ms
Wall time: 462 ms


In [342]:
df_data

,year,month,region,oktmo,SUBJ,clinic,oid,nameShort,depart,branch,...,usl_doctor_specialty,POSTNAME_y,usl_clinic,usl_depart,sl_id,code_usl,kol_usl,yet_sum,size,visit_count
0,2023,1,99,99000000000,Еврейская АО,790001,1.2.643.5.1.13.13.12.2.79.8850,ОГБУЗ Областная больница,79000100,nan,...,24,врач-инфекционист,790001,79000100,1,1,1,nan,1,1
1,2023,1,99,99000000000,Еврейская АО,790001,1.2.643.5.1.13.13.12.2.79.8850,ОГБУЗ Областная больница,79000100,nan,...,24,врач-инфекционист,790001,79000100,1,3,1,nan,1,3
2,2023,1,99,99000000000,Еврейская АО,790001,1.2.643.5.1.13.13.12.2.79.8850,ОГБУЗ Областная больница,79000100,nan,...,24,врач-инфекционист,790001,79000100,1,3,1,nan,1,3
3,2023,1,99,99000000000,Еврейская АО,790001,1.2.643.5.1.13.13.12.2.79.8850,ОГБУЗ Областная больница,79000100,nan,...,24,врач-инфекционист,790001,79000100,1,1,1,nan,1,1
4,2023,1,99,99000000000,Еврейская АО,790001,1.2.643.5.1.13.13.12.2.79.8850,ОГБУЗ Областная больница,79000100,nan,...,24,врач-инфекционист,790001,79000100,1,1,1,nan,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25405,2023,2,99,99000000000,Еврейская АО,790021,1.2.643.5.1.13.13.12.2.79.8838,"ОГБУЗ ""Онкологический диспансер""",79002100,nan,...,206,фельдшер,790021,79002100,20,1,1,nan,1,20
25406,2023,2,99,99000000000,Еврейская АО,790021,1.2.643.5.1.13.13.12.2.79.8838,"ОГБУЗ ""Онкологический диспансер""",79002100,nan,...,206,фельдшер,790021,79002100,4,1,1,nan,1,4
25407,2023,2,99,99000000000,Еврейская АО,790021,1.2.643.5.1.13.13.12.2.79.8838,"ОГБУЗ ""Онкологический диспансер""",79002100,nan,...,206,фельдшер,790021,79002100,117,1,1,nan,1,117
25408,2023,2,99,99000000000,Еврейская АО,790021,1.2.643.5.1.13.13.12.2.79.8838,"ОГБУЗ ""Онкологический диспансер""",79002100,nan,...,206,фельдшер,790021,79002100,1,1,1,nan,1,1


In [343]:
df_data.shape

(25410, 32)

## 4. Контроль результатов

### Не расшифрованные данные:

In [344]:
# не расшифрованы данные о субъекте
df_data.loc[df_data[df_data["SUBJ"].isna()].index]

,year,month,region,oktmo,SUBJ,clinic,oid,nameShort,depart,branch,...,usl_doctor_specialty,POSTNAME_y,usl_clinic,usl_depart,sl_id,code_usl,kol_usl,yet_sum,size,visit_count


In [345]:
# не расшифрованы данные о наименовании медучреждения
df_data.loc[df_data[df_data["nameShort"].isna()].index]

,year,month,region,oktmo,SUBJ,clinic,oid,nameShort,depart,branch,...,usl_doctor_specialty,POSTNAME_y,usl_clinic,usl_depart,sl_id,code_usl,kol_usl,yet_sum,size,visit_count


In [346]:
# не расшифрованы данные о специальности врача
df_data.loc[df_data[df_data["POSTNAME_x"].isna()].index]

,year,month,region,oktmo,SUBJ,clinic,oid,nameShort,depart,branch,...,usl_doctor_specialty,POSTNAME_y,usl_clinic,usl_depart,sl_id,code_usl,kol_usl,yet_sum,size,visit_count


In [347]:
# не расшифрованы данные о профиле врача
df_data.loc[df_data[df_data["PRNAME"].isna()].index]

,year,month,region,oktmo,SUBJ,clinic,oid,nameShort,depart,branch,...,usl_doctor_specialty,POSTNAME_y,usl_clinic,usl_depart,sl_id,code_usl,kol_usl,yet_sum,size,visit_count


In [348]:
# не расшифрованы данные о специальности врача, оказавшего услугу
df_data.loc[df_data[df_data["POSTNAME_y"].isna()].index]

,year,month,region,oktmo,SUBJ,clinic,oid,nameShort,depart,branch,...,usl_doctor_specialty,POSTNAME_y,usl_clinic,usl_depart,sl_id,code_usl,kol_usl,yet_sum,size,visit_count


In [349]:
# не расшифрованы данные об условиях оказания помощи
df_data.loc[df_data[df_data["UMPNAME"].isna()].index]

,year,month,region,oktmo,SUBJ,clinic,oid,nameShort,depart,branch,...,usl_doctor_specialty,POSTNAME_y,usl_clinic,usl_depart,sl_id,code_usl,kol_usl,yet_sum,size,visit_count


In [350]:
# не расшифрованы данные оцели посещения
df_data.loc[df_data[df_data["N_PC"].isna()].index]

,year,month,region,oktmo,SUBJ,clinic,oid,nameShort,depart,branch,...,usl_doctor_specialty,POSTNAME_y,usl_clinic,usl_depart,sl_id,code_usl,kol_usl,yet_sum,size,visit_count


In [351]:
# формат данных
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25410 entries, 0 to 25409
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   year                  25410 non-null  int64  
 1   month                 25410 non-null  int64  
 2   region                25410 non-null  int64  
 3   oktmo                 25410 non-null  int64  
 4   SUBJ                  25410 non-null  object 
 5   clinic                25410 non-null  int64  
 6   oid                   25410 non-null  object 
 7   nameShort             25410 non-null  object 
 8   depart                25410 non-null  int64  
 9   branch                25410 non-null  object 
 10  doctor_specialty      25410 non-null  int64  
 11  POSTNAME_x            25410 non-null  object 
 12  doctor_profile        25410 non-null  int64  
 13  PRNAME                25410 non-null  object 
 14  terms                 25410 non-null  int64  
 15  UMPNAME            

In [352]:
# пустые количества посещений
df_data[np.isnan(df_data["visit_count"])]

,year,month,region,oktmo,SUBJ,clinic,oid,nameShort,depart,branch,...,usl_doctor_specialty,POSTNAME_y,usl_clinic,usl_depart,sl_id,code_usl,kol_usl,yet_sum,size,visit_count


In [353]:
# не определённые количества посещений
df_data[df_data["visit_count"] == "не определено"]

,year,month,region,oktmo,SUBJ,clinic,oid,nameShort,depart,branch,...,usl_doctor_specialty,POSTNAME_y,usl_clinic,usl_depart,sl_id,code_usl,kol_usl,yet_sum,size,visit_count


In [354]:
# сохранение результатов на диск
df_data.to_csv("/home/sk27/work/med/data/result.csv", encoding="utf-8-sig")